In [1]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
pd.options.mode.chained_assignment = None

In [17]:
import pickle
with open('one_obj_to_rule_them_all.pkl', 'rb') as p:
    obj = pickle.load(p)

data = obj['data']
meta_snowex = obj['meta']['snowex']
meta_awdb = obj['meta']['awdb']

meta_snowex

,site_name,site_id,latitude,longitude,geometry
0,Grand Mesa,2C12,39.053236,-108.200765,POINT (-108.20076 39.05324)
1,Grand Mesa,6S26,39.018401,-108.167333,POINT (-108.16733 39.01840)
2,Grand Mesa,5N32,39.030444,-108.171691,POINT (-108.17169 39.03044)
3,Grand Mesa,FL1B,39.021928,-108.174168,POINT (-108.17417 39.02193)
4,Grand Mesa,5N24,39.032945,-108.185684,POINT (-108.18568 39.03295)
...,...,...,...,...,...
195,Sagehen Creek,Forest,39.429680,-120.242110,POINT (-120.24211 39.42968)
196,Sagehen Creek,Open,39.430410,-120.239810,POINT (-120.23981 39.43041)
197,Sagehen Creek,Tower 4,39.422160,-120.298980,POINT (-120.29898 39.42216)
198,Senator Beck,Senator Beck,37.907020,-107.726230,POINT (-107.72623 37.90702)


In [5]:
from folium.plugins import MarkerCluster
points_snowex = MarkerCluster(name='SnowEx')
points_awdb = MarkerCluster(name='AWDB')


# # Create color column to differentiate points
# colors = ['red' if el == '36701' else 'green' for el in points['FCode']]
# points['color'] = colors

# Loop through all points and add as markers to cluster
for ind in meta_snowex.index:
    fcode_type = points.loc[ind, 'type']
    huc8 = points.loc[ind, 'HUC8']
    folium.Marker(
        # Need to extract lat/lon from geometry
        # Marker expects ordered pair, not GeoJSON
        location=(points.loc[ind, 'geometry'].y, points.loc[ind, 'geometry'].x), 
        tooltip=folium.features.Tooltip(
            text=f'Type: {fcode_type}\nHUC8: {huc8}',
            style=('background-color: white; \
             color: #333333; \
             font-family: arial; \
             font-size: 12px; padding: 10px; white-space: pre')),
        icon=folium.Icon(
            color=points.loc[ind, 'color'], 
            icon_color='white', 
            icon='circle', angle=0, prefix='fa')).add_to(cluster)

# Create map object
huc_map = folium.Map(location=[46, -110], zoom_start=7, tiles=None, 
                     control_scale=True)
# Add two background layers
folium.TileLayer('Stamen terrain', name='Terrain basemap', control=True).add_to(huc_map)
folium.TileLayer('CartoDB positron', name='Light basemap', control=True).add_to(huc_map)

# # Create tooltip using HUC8 DataFrame
# huc_tooltip = folium.GeoJsonTooltip(
#     fields=['HUC8','Name','AreaSqKm','num_gauges','num_dams'],
#     aliases=['HUC8: ','Name: ','Area (sq km): ','Number of streamgauges: ','Number of dams: '])

# # Create a function that returns the style of highlight we want
# def highlighter(x): 
#     return {'fillColor': '#000000', 
#             'color':'#000000', 
#             'fillOpacity': 0.50, 
#             'weight': 0.1}

# # Create a style function
# # First, create a colormap based on ColorBrewer naming scheme
# from branca.colormap import linear as linear_cm
# cmap = linear_cm.YlGnBu_09.scale(min(huc_viz['AreaSqKm']),
#                                  max(huc_viz['AreaSqKm']))
# # Style function
# def styler(x):
#     return {'fillColor': cmap(x['properties']['AreaSqKm']), 
#             'color':'#000000', 
#             'fillOpacity': 1.0, 
#             'weight': 0.5}

# # Add HUC8 polygons as GeoJson layer
# huc_layer = folium.GeoJson(huc_viz, 
#                            tooltip=huc_tooltip, 
#                            style_function=styler,
#                            highlight_function=highlighter,
#                            name='HUC8')
# huc_map.add_child(huc_layer)
# #huc_map.keep_in_front(huc_layer)

# # Add marker cluster layer
# cluster.add_to(huc_map)

# # Add color bar to map
# cmap.caption = 'Watershed area [sq km]'
# cmap.add_to(huc_map)

# Layer control to switch between layers
huc_map.add_child(folium.LayerControl())
huc_map